In [1]:
import pandas as pd
import os
import datetime
import gc
import numpy as np

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Bruce_2019Q1_by_Platform_Audience_20190808'

In [2]:
excel_file_KL=pd.ExcelFile("./BL 2019Q1 Media Delivery by Reward Segment KL 20190807.xlsx")
excel_file_KL.sheet_names

['Audience report (7)',
 'Segment Key',
 'TTD_YT',
 'TTD Partners',
 'YouTube',
 'Facebook',
 'Pinterest',
 'Twitter',
 'Pandora',
 'serta raw',
 'Sheet3']

In [3]:
df_mapping_segment_platform_seg=excel_file_KL.parse("Segment Key",dtype=str)
df_mapping_segment_platform_seg.shape

(90, 6)

In [4]:
df_all_partner_audience_seg=pd.DataFrame()
for col in df_mapping_segment_platform_seg.columns.tolist():
    if "Segment" in col:
        df=df_mapping_segment_platform_seg[['Partner',"Audience Name"]+[col]].rename(columns={col:"segment"})
        df_all_partner_audience_seg=df_all_partner_audience_seg.append(df)

df_all_partner_audience_seg=df_all_partner_audience_seg[df_all_partner_audience_seg['segment']!="nan"]    
df_all_partner_audience_seg=df_all_partner_audience_seg[pd.notnull(df_all_partner_audience_seg['segment'])]    
df_all_partner_audience_seg['cleaned_segment']=df_all_partner_audience_seg['segment'].apply(lambda x: x.replace("segment=","").strip())
df_all_partner_audience_seg=df_all_partner_audience_seg.rename(columns={"Audience Name":"Audience"})

In [5]:
tab_list=["TTD_YT","Facebook","Pinterest","Twitter","Pandora"]

df_TTD_YT=excel_file_KL.parse("TTD_YT",skiprows=2,dtype=str).rename(columns={"Row Labels":"Audience","Sum of Impressions ":"imprs","Sum of Media Cost":"cost"})
df_TTD_YT['Partner']="TTD_YT"

df_Facebook=excel_file_KL.parse("Facebook",dtype=str).rename(columns={"Ad Sets":"Audience","Impressions":"imprs","Media Cost":"cost"})
df_Facebook['Partner']="Facebook"

df_Pinterest=excel_file_KL.parse("Pinterest",dtype=str).rename(columns={"Ad group name":"Audience","Total impressions":"imprs","Media Cost":"cost"})
df_Pinterest['Partner']="Pinterest"

df_Twitter=excel_file_KL.parse("Twitter",dtype=str).rename(columns={"Ad Sets":"Audience","Impressions":"imprs","Media Cost":"cost"})
df_Twitter['Partner']="Twitter"

df_Pandora=excel_file_KL.parse("Pandora",dtype=str).rename(columns={"Placement":"Audience","IMPs":"imprs","Media Cost":"cost"})
df_Pandora['Partner']="Pandora"



In [6]:
df_all_media=pd.concat([df_TTD_YT,df_Facebook,df_Pinterest,df_Twitter,df_Pandora])
df_all_media['imprs']=df_all_media['imprs'].astype(float)
df_all_media['cost']=df_all_media['cost'].astype(float)


In [7]:
df_all_media.head(10)

,Audience,imprs,cost,Partner
0,Patio AP,1077126.0,18242.000000,TTD_YT
1,Patio AS,288974.0,5032.280000,TTD_YT
2,Patio Quad 2 AP,519024.0,9180.080000,TTD_YT
3,Patio Quad 2 AS,94307.0,1656.410000,TTD_YT
4,Patio Quad 3 AP,476265.0,8091.810000,TTD_YT
5,Patio Quad 3 AS,82673.0,1427.020000,TTD_YT
6,Patio Quad 4 AP,678249.0,10582.660000,TTD_YT
7,segment=Experian_T_Holiday Shoppers,700977.0,7020.199710,TTD_YT
8,segment=Experian_T_Holiday_Shoppers,375220.0,5376.030000,TTD_YT
9,segment=Experian_T_Home Focused,995168.0,10206.479124,TTD_YT


In [10]:
df_output=pd.merge(df_all_partner_audience_seg,df_all_media,on=["Partner","Audience"],how="left")

In [16]:
df_output[pd.isnull(df_output['imprs'])]

,Partner,Audience,segment,cleaned_segment,imprs,cost
73,"Pinterest, FB, Twitter",P_Grow_LR_NewRewards,New Rewards Sign Ups,New Rewards Sign Ups,NaN,NaN
74,"Pinterest, FB, Twitter",Quad1_P_Defend_LR_Active,segment=T_Quadrant I_P_H_2019Q1,T_Quadrant I_P_H_2019Q1,NaN,NaN
75,"Pinterest, FB, Twitter",Quad1_S_Defend_LR_Active,segment=T_Quadrant I_S_H_2019Q1,T_Quadrant I_S_H_2019Q1,NaN,NaN
76,"Pinterest, FB, Twitter",Quad2_P_Defend_LR_Active,segment=T_Quadrant II_P_H_2019Q1,T_Quadrant II_P_H_2019Q1,NaN,NaN
77,"Pinterest, FB, Twitter",Quad2_S_Defend_LR_Active,segment=T_Quadrant II_S_H_2019Q1,T_Quadrant II_S_H_2019Q1,NaN,NaN
78,"Pinterest, FB, Twitter",Quad3_P_Defend_LR_Active,segment=T_Quadrant III_P_H_2019Q1,T_Quadrant III_P_H_2019Q1,NaN,NaN
79,"Pinterest, FB, Twitter",Quad3_P_Grow_LR_Lapsed,segment=T_Quadrant III_P_L_2019Q1,T_Quadrant III_P_L_2019Q1,NaN,NaN
80,"Pinterest, FB, Twitter",Quad3_S_Defend_LR_Active,segment=T_Quadrant III_S_H_2019Q1,T_Quadrant III_S_H_2019Q1,NaN,NaN
81,"Pinterest, FB, Twitter",Quad3_S_Grow_LR_Lapsed,segment=T_Quadrant III_S_L_2019Q1,T_Quadrant III_S_L_2019Q1,NaN,NaN
82,"Pinterest, FB, Twitter",Quad4_P_Grow_LR_Active,segment=T_Quadrant IV_P_H_2019Q1,T_Quadrant IV_P_H_2019Q1,NaN,NaN
